# Recommend Projects to Users

In [1]:
from keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
np.random.seed(0)
from time import gmtime, strftime
from keras.models import Model
from scipy.sparse import vstack

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.evaluate import evaluate
from src.recommender import Recommender
import src.load_data as load_data

Using TensorFlow backend.


In [2]:
# Load the autoencoder to reduce the dimensionality of our TF-IDF vectors
model = load_model('autoencoder.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
# Load the proejct data
train_projects, train_x, test_projects, test_x, train_project_ids, test_project_ids = load_data.load_projects()
train_x_projects = np.array(train_projects, dtype=np.int32).reshape(len(train_projects), 1)
test_x_projects = np.array(test_projects, dtype=np.int32).reshape(len(test_projects), 1)

# Prepare our variables to be used in recommendation
x = vstack([train_x, test_x])
x_projects = train_projects + test_projects
x_project_ids = train_project_ids + test_project_ids

# Load the users projects
users_projects = load_data.load_user()

In [4]:
# Create a model that we will use to extract the embedding layer output
embed_model = Model(inputs=model.input, outputs=model.get_layer('embedding_layer').output)

# The dimension that the TF-IDF vector will be reduced to
embedding_size = model.get_layer('embedding_layer').output_shape[2]

# Embed our TF-IDF Vectors for all Projects
embeddings = embed_model.predict(x=[x, np.array(x_projects, dtype=np.int32).reshape(len(x_projects), 1)]).reshape(len(x_projects), embedding_size)

### Make Recommendations

In [5]:
# Initialise our recommender class
rec = Recommender(x_project_ids)

# Calculate our cosine similarity Matrix between all projects
similarity_matrix = rec.similarity(embeddings)

In [6]:
user_idx = 0
user_projects = users_projects[user_idx].todense()
user_projects = np.array((user_projects.T))
done_projects, top_projects = rec.top_projects(similarity_matrix, user_projects)

In [8]:
y_true, y_pred = rec.predictions(user_projects, done_projects, top_projects) 

precision, recall = evaluate(y_true, y_pred)

In [9]:
print('--------------------------------')
print('Model Results')
print('Precision: ' + str(precision))
print('Recall: ' + str(recall))
print('--------------------------------')

--------------------------------
Model Results
Precision: 0.0
Recall: 0.0
--------------------------------
